In [ ]:
import math
from collections import defaultdict
from chempy import ReactionSystem
from chempy.units import (
    default_units as u,
    SI_base_registry as ureg
)
from chempy.kinetics.ode import get_odesys
from chempy.kinetics.rates import SinTemp
%matplotlib inline

In [ ]:
rsys = ReactionSystem.from_string("""
2 HNO2 -> H2O + NO + NO2; EyringParam(dH=85e3*J/mol, dS=10*J/K/mol)
2 NO2 -> N2O4; EyringParam(dH=70e3*J/mol, dS=20*J/K/mol)
"""
)

In [ ]:
st = SinTemp(unique_keys='Tbase Tamp Tangvel Tphase'.split())

In [ ]:
odesys, extra = get_odesys(rsys, include_params=False, substitutions={
    'temperature': st}, unit_registry=ureg)

In [ ]:
init_conc = defaultdict(lambda: 0*u.M, HNO2=1*u.M, H2O=55*u.M)
params = dict(
    Tbase=300*u.K,
    Tamp=10*u.K,
    Tangvel=2*math.pi/(10*u.s),
    Tphase=-math.pi/2
)
duration = 60*u.s

In [ ]:
def integrate_and_plot(system):
    result = system.integrate(duration, init_conc, params, integrator='cvode', nsteps=2000)
    result.plot(names='NO HNO2 N2O4 NO2'.split())
    print({k: v for k, v in sorted(result.info.items()) if not k.startswith('internal')})

In [ ]:
integrate_and_plot(odesys)

In [ ]:
autsys = odesys.as_autonomous()

In [ ]:
autsys.exprs

In [ ]:
integrate_and_plot(autsys)